# Project 3


## Importing Modules and staring local db engine

In [1]:
#Importing modules

import boto3

from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector
import importlib
import json
import numpy as np
import pandas as pd
import re
import requests
from sqlalchemy import create_engine
from sqlalchemy import inspect
import tabula
import yaml


#starting local database

#read creds
uploader = DatabaseConnector()
ml_yaml = 'ml_dbs.yaml'
uploader.read_db_creds(ml_yaml)

# initialising and returning an sqlalchemy database engine.

uploading = uploader.init_db_engine_postgresql()



Credentials loaded sucessfully!!
Connection successful!


## RDS ETL: User


In [ ]:
### 1. Getting data from the RDS database

## read credentials
connector_rds = DatabaseConnector()
ai_core_yaml = 'db_creds.yaml'
connector_rds.read_db_creds(ai_core_yaml)

## initialising and returning an sqlalchemy database engine.
connection_rds = connector_rds.init_db_engine()

## Reading the data from the RDS database
connector_tables_rds = connector_rds.list_db_tables()


### 2. Extracting the 'legacy_users' table to a pandas DataFrame.
user_table_rds = 'legacy_users'
extractor_rds = DataExtractor(connection_rds)
user_df_rds = extractor_rds.read_rds_table(user_table_rds)
print(user_df_rds.head(5))
print(user_df_rds.info())



In [ ]:
### 3.Cleaning data
clean_data_rds = DataCleaning()
clean_df_rds = clean_data_rds.clean_user_data(user_df_rds)
clean_dob = clean_data_rds.date_formatting(clean_df_rds, 'date_of_birth')

clean_join_date = clean_data_rds.date_formatting(clean_dob,'join_date')
clean_email = clean_data_rds.email_clean(clean_join_date, 'email_address')
clean_first_name = clean_data_rds.gibberish_clean(clean_email, 'first_name')
clean_last_name = clean_data_rds.gibberish_clean(clean_first_name, 'last_name')
clean_dim_users = clean_data_rds.null_replace(clean_last_name)


In [ ]:
### 4. Loading the data
#load clean rds dataframe into local database with a new table
uploader.upload_to_db(uploading, clean_df_rds, 'dim_users')

## PDF ETL: Card Details

In [ ]:
### 1.Getting the data from PDF & load in df
connector_pdf = DatabaseConnector()
extractor_pdf = DataExtractor(connector_pdf)

link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf"
extraction_pdf = extractor_pdf.retrieve_pdf_data(link)
print(extraction_pdf.head(5))
print(extraction_pdf.info())

### 2.Clean the pdf df

clean_pdf = DataCleaning()

clean_df_pdf =clean_pdf.clean_card_data(extraction_pdf)

### 3. load into db

uploader.upload_to_db(uploading, clean_df_pdf, 'dim_card_details')

## API ETL:  Store







In [2]:
### 1. Getting Data from API


headers = {
    'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'
}

number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
store_details_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}'

connector_api = DatabaseConnector()
extractor_api = DataExtractor(connector_api)


number_of_stores = extractor_api.list_number_of_stores(number_of_stores_endpoint, headers)
print(number_of_stores)


### 2. Extracting the table to a pandas DataFrame
retrieve_store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/2'

stores_df = extractor_api.retrieve_stores_data(retrieve_store_endpoint, headers, 2)


{'statusCode': 200, 'number_stores': 451}


In [ ]:
headers = {
    'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'
}

number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
store_details_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{store_number}'

extractor_api = DataExtractor()
number_of_stores = extractor_api.list_number_of_stores(number_of_stores_endpoint, headers)
print(f"Number of stores: {number_of_stores}")

stores_df = extractor_api.retrieve_stores_data(store_details_endpoint, headers, number_of_stores)
print(stores_df.head())

cleaner = DataCleaning()
clean_stores_df = cleaner._clean_store_data(stores_df)
print(clean_stores_df.head())

uploader = DataUploader()
uploader.upload_to_db(clean_stores_df, 'dim_store_details')

In [ ]:
### 3.Cleaning data

clean_stores_api = DataCleaning()


clean_store_date = clean_stores_api.date_formatting(stores_df, 'opening_date')


In [ ]:
### 4. Loading the data
uploader.upload_to_db(uploading,clean_store_date, 'dim_store_details')

## S3 ETL: Products


In [ ]:
### 1. Getting data

connector_s3 = DatabaseConnector()
extractor_s3 = DataExtractor(connector_s3)


### 2. Putting into pd df

bucket = 'data-handling-public'
object_key = "products.csv"
pathway = '/Users/student/AICORE/AWS/Project_3/products.csv'

extraction_s3 = extractor_s3.download_csv_from_s3(bucket, object_key, pathway)
print(extraction_s3.head(20))
print(extraction_s3.info())

### 3. Cleaning data

clean_s3 = DataCleaning()

clean_df_s3 = clean_s3.called_clean_store_data(extraction_s3)


clean_df_s3 = clean_s3.called_clean_store_data(extraction_s3)


converted_df_s3 = clean_s3.convert_product_weights(clean_df_s3)
print(converted_df_s3.head())


clean_df_s3 = clean_s3.clean_products_data(converted_df_s3)


### 4. Loading into local db
uploader.upload_to_db(uploading, clean_df_s3, 'dim_products')


## AWS DB ETL: Orders

In [ ]:
connector_tables_rds = connector_rds.list_db_tables()

#'orders_table'


In [ ]:
orders_table_rds = 'orders_table'

extractor_rds = DataExtractor(connection_rds)

orders_df_rds = extractor_rds.read_rds_table(orders_table_rds)
print(orders_df_rds.head(5))
print(orders_df_rds.info())

#cleaning the orders table

clean_ordersdf_rds = clean_data_rds.clean_user_data(orders_df_rds)

## dropping columns

ordersdf_rds = clean_ordersdf_rds.drop(['first_name','last_name','1' ], axis = 1)
print(ordersdf_rds.info())

#uploading the updated orders table into the local db
uploader.upload_to_db(uploading, ordersdf_rds, 'orders_table')


## JSON S3 ETL: Date Details

In [3]:
#Preparing the extration

connector_json_s3 = DatabaseConnector()
extractor_json_s3 = DataExtractor(connector_json_s3)
bucket = 'data-handling-public'
object_key_json = "date_details.json"

#Get the object

extraction_json_s3 = extractor_json_s3.extract_json_from_s3(bucket, object_key_json)
print(extraction_json_s3.head(20))
print(extraction_json_s3.info())

#Clean the data
clean_json_s3 = DataCleaning()

clean_df_json_s3 = clean_json_s3.called_clean_store_data(extraction_json_s3)
clean_df_date = clean_json_s3.null_replace(extraction_json_s3)

#upload to local db
uploader.upload_to_db(uploading, clean_df_json_s3, 'dim_date_times')

/Users/student/AICORE/AWS/Project_3/data_extraction.py:68: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_s3 = pd.read_json(content)


   timestamp month  year day time_period                             date_uuid
0   22:00:06     9  2012  19     Evening  3b7ca996-37f9-433f-b6d0-ce8391b615ad
1   22:44:06     2  1997  10     Evening  adc86836-6c35-49ca-bb0d-65b6507a00fa
2   10:05:37     4  1994  15     Morning  5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3   17:29:27    11  2001   6      Midday  1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4   22:40:33    12  2015  31     Evening  dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
5   02:03:25     8  2002   2  Late_Hours  cea14ebe-06b2-4cee-8394-972101419375
6   20:41:58     1  1993  14     Evening  5b72f016-349c-4649-aea0-bcc4fd1b9ee6
7   22:36:19     3  2006  15     Evening  5d598d23-ee71-4fc9-a895-77fd3891a7c4
8   18:06:15    12  1994  14     Evening  ad50dbd6-b736-4ffe-a745-6ebed29e4cdf
9   16:12:22    11  1994   1      Midday  92fc92a3-c4a0-49a4-a2ee-40a37f9d11dc
10  21:21:31    11  1994  21     Evening  21451b39-3bac-4106-b238-8d78c6a08692
11  19:23:21     8  2004   3     Evening  08f17ff8-8